In [1]:
from __future__ import division, print_function, absolute_import

import tflearn

/home/shared/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Use the retry module or similar alternatives.


In [9]:
# Residual blocks
# 32 layers: n=5, 56 layers: n=9, 110 layers: n=18
n = 5

# Data loading
# should create the HDF5 files first...
import h5py
h5f = h5py.File('train.h5', 'r')
X = h5f['X']
Y = h5f['Y']

h5f_test = h5py.File('test.h5', 'r')
testX = h5f_test['X']
testY = h5f_test['Y']

OSError: Unable to open file (unable to open file: name = 'train.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

In [4]:
# Real-time data preprocessing
img_prep = tflearn.ImagePreprocessing()
# why is this mean hardcoded?
img_prep.add_featurewise_zero_center(per_channel=True, mean=[0.53990436 , 0.4405486  , 0.39328504])


# Real-time data augmentation
img_aug = tflearn.ImageAugmentation()
img_aug.add_random_flip_leftright()
img_aug.add_random_crop([49, 49], padding=4)

In [6]:
n = 5
# Building Residual Network
net = tflearn.input_data(shape=[None, 49, 49, 3],
                         data_preprocessing=img_prep,
                         data_augmentation=img_aug)
net = tflearn.conv_2d(net, 16, 3, regularizer='L2', weight_decay=0.0001)
net = tflearn.resnext_block(net, n, 16, 32)
net = tflearn.resnext_block(net, 1, 32, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 32, 32)
net = tflearn.resnext_block(net, 1, 64, 32, downsample=True)
net = tflearn.resnext_block(net, n-1, 64, 32)
net = tflearn.batch_normalization(net)
net = tflearn.activation(net, 'relu')
net = tflearn.global_avg_pool(net)

In [7]:
# Regression
net = tflearn.fully_connected(net, 11, activation='softmax')
opt = tflearn.Momentum(0.1, lr_decay=0.1, decay_step=32000, staircase=True)
net = tflearn.regression(net, optimizer=opt,
                         loss='categorical_crossentropy')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [8]:
model = tflearn.DNN(net, checkpoint_path='Snapshots/model_resnext',
                    max_checkpoints=10, tensorboard_verbose=0, tensorboard_dir='Logs/',
                    clip_gradients=0.)

In [ ]:
model.fit(X, Y, n_epoch=200, validation_set=(testX, testY),
          snapshot_epoch=False, snapshot_step=200,
          show_metric=True, batch_size=128, shuffle=True,
          run_id='resnext')